##Amazon web scrapping

In [ ]:
#microsoft edge
!pip install msedge-selenium-tools selenium==3.141
from msedge.selenium_tools import Edge, EdgeOptions

#Driver setup

In [ ]:
import csv
from bs4 import BeautifulSoup
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


#Query generator

In [26]:
def get_url(search_term):
  """Generate a url from search term"""
  template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
  search_term = search_term.replace(' ','+')
  url = template.format(search_term)
  #https://www.amazon.com/s?k=curved+monitor&page=3&qid=1617027061&ref=sr_pg_3
  url +='&page{}'
  return url
test_url = get_url('curve monitor')
print(test_url)

https://www.amazon.com/s?k=curve+monitor&ref=nb_sb_noss_1&page{}


#return results

In [40]:
def extract_record(item):
  """extract and return data from a single record"""
  atag = item.h2.a
  description = atag.text.strip()
  item_url = 'https://www.amazon.com/' + atag.get('href')
  try:
    price = item.find('span','a-offscreen').text
  except AttributeError:
    return
  try:
    rating = item.i.text.split()[0]
    review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
  except AttributeError:
    rating = ''
    review_count = ''
  result = [description,price,rating,review_count,item_url]
  return result

#Write in CSV

In [41]:
def write_to_csv(records):
  with open('amazon_results.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Description','Price','Rating','Review','Url'])
    writer.writerows(records)

Putting it together

In [42]:
def main(search_term):
  driver = webdriver.Chrome(options=options)
  records = []
  url =get_url(search_term)
  for page in range(1,21):
    driver.get(url.format(page))
    soup = BeautifulSoup(driver.page_source,'lxml')
    results = soup.find_all('div',{'data-component-type':'s-search-result'})
    for item in results:
      single_record =extract_record(item)
      if single_record:
        records.append(single_record)
  driver.close()
  write_to_csv(records)
  print(len(records))


In [43]:
main('curved monitor')

440
